In [ ]:
import torch
height = width = 16

height_gap = 2 / (height - 1)
width_gap = 2 / (width - 1)
height, width = torch.meshgrid([torch.range(-1, 1, height_gap), torch.Tensor(1)])

print(torch.range(-1, 1, height_gap))
print(height)

print(width)

lr_identity = torch.stack([width, height])


print(lr_identity.shape)

In [ ]:
a = {}
a['b'] = 2
a['c'] = 10

for k,v in a.items():
    print(k,v)

In [ ]:
a = [1,2,3,4]
def func(a):
    yield a[0], a[1], a[2], a[3]
    
for i in func(a):
    print(i)

In [ ]:
a = [1,2,3,4]
print(set(a))
def func(a):
    yield a[0]
    for i in range(len(a[1:])):
        yield a[i]
    
for i in func(a):
    print(i)

In [ ]:
a = True
b = True

a and b


import random
a = random.random()
print(a < 0.5)

b = b and random.random() < 0.5

In [ ]:
import torch
idx_data = 0
a = torch.rand([5,2,7])
print(a)
b, epoch_idx = a.max(0)
best, frame_idx = b.max(1)
print(epoch_idx)
print(best)
print(frame_idx)


best_frame_idx = frame_idx[idx_data]
print(best_frame_idx)
best_epoch_idx = epoch_idx[idx_data, frame_idx[idx_data]]
print(best_epoch_idx)

print(a[best_epoch_idx, idx_data, best_frame_idx])

In [ ]:
a = '10'

try:
    print("frame" + str(a))
except:
    raise TypeError("a should be int type")

In [ ]:
import torch
a = torch.rand([5,2,7])

print(a.shape[0])

In [ ]:
from tqdm import tqdm
import numpy as np
from time import sleep
for idx, i in enumerate(tqdm(np.arange(100))):
    print(idx)
    sleep(0.01)

In [ ]:
import torch

a = torch.tensor([3,4,5])

print(a)

print("a is {}".format(a))



In [ ]:
import numpy as np
import os

TAG_FLOAT = 202021.25
import torch.nn as nn
def read_flow(file):

    assert type(file) is str, "file is not str %r" % str(file)
    assert os.path.isfile(file) is True, "file does not exist %r" % str(file)
    assert file[-4:] == '.flo', "file ending is not .flo %r" % file[-4:]
    f = open(file,'rb')
    flo_number = np.fromfile(f, np.float32, count=1)[0]
    assert flo_number == TAG_FLOAT, 'Flow number %r incorrect. Invalid .flo file' % flo_number
    w = np.fromfile(f, np.int32, count=1)
    h = np.fromfile(f, np.int32, count=1)
    data = np.fromfile(f, np.float32, count=2*w[0]*h[0])
    #if error try: data = np.fromfile(f, np.float32, count=2*w*h)
    # Reshape data into 3D array (columns, rows, bands)
    flow = np.resize(data, (int(h), int(w), 2))
    f.close()

    return flow

def warp(x, flo):
    """
    warp an image/tensor (im2) back to im1, according to the optical flow.
    Code heavily inspired by
    x: [B, C, H, W] (im2)
    flo: [B, 2, H, W] flow
    """
    B, C, H, W = x.size()
    # mesh grid
    xx = torch.arange(0, W).view(1, -1).repeat(H, 1)
    yy = torch.arange(0, H).view(-1, 1).repeat(1, W)
    xx = xx.view(1, 1, H, W).repeat(B, 1, 1, 1)
    yy = yy.view(1, 1, H, W).repeat(B, 1, 1, 1)
    grid = torch.cat((xx, yy), 1).float()
    #print(grid)
    grid = grid
    vgrid = grid + flo

    # scale grid to [-1,1]
    vgrid[:, 0, :, :] = 2.0*vgrid[:, 0, :, :]/max(W-1, 1)-1.0
    vgrid[:, 1, :, :] = 2.0*vgrid[:, 1, :, :]/max(H-1, 1)-1.0
    vgrid = vgrid.permute(0, 2, 3, 1)
    output = nn.functional.grid_sample(x, vgrid)
    # Define a first mask containing pixels that wasn't properly interpolated
    mask = torch.autograd.Variable(torch.ones(x.size()))
    mask = nn.functional.grid_sample(mask, vgrid)
    mask[mask < 0.9999] = 0
    mask[mask > 0] = 1

    return output, mask


filename = "/home/lrh/git/blind-denoising/tvl1flow/of/tvl1_target.flo"

of = read_flow(filename)
print(of.max())
print(of.min())

print(of.shape)

of = torch.tensor(of).unsqueeze(0).permute(0,3,1,2)

image_path = "/home/lrh/git/blind-denoising/tvl1flow/image/target/im3.png"
import imageio
import torch
image_np = cv2.imread(image_path)
image_tensor = torch.tensor(image_np).type(torch.FloatTensor).unsqueeze(0).permute(0,3,1,2)
print(image_tensor.shape)

o, m = warp(image_tensor, -of)

from matplotlib import pyplot as plt

target_path = "/home/lrh/git/blind-denoising/tvl1flow/image/target/im4.png"
image_target = cv2.imread(target_path)


plt.figure(figsize=(12,8))
plt.subplot(221)
plt.imshow(image_np)

plt.subplot(222)
plt.imshow(image_target)

plt.subplot(223)
plt.imshow(o[0].permute(1,2,0).numpy() / 255.0)

plt.show()

In [ ]:
import cv2
import numpy as np
from imageio import imread
img_file = '/home/lrh/git/blind-denoising/tvl1flow/image/target/im3.png'
img2_file = '/home/lrh/git/blind-denoising/tvl1flow/image/target/im4.png'

prev_frame =  cv2.imread(img_file)
next_frame = cv2.imread(img2_file)

flow_file = "/home/lrh/git/blind-denoising/tvl1flow/of/tvl1_target.flo"
flow = read_flow(flow_file)

def warp_flow(img, flow):
    h, w = flow.shape[:2]
    flow = - flow
    flow[:,:,0] += np.arange(w)
    flow[:,:,1] += np.arange(h)[:,np.newaxis]
    res = cv2.remap(img, flow, None, cv2.INTER_LINEAR)
    return res

im2w = warp_flow(prev_frame, flow)


plt.figure(figsize=(12,8))
plt.subplot(221)
plt.imshow(prev_frame / 255.0)

plt.subplot(222)
plt.imshow(next_frame/255.0)

plt.subplot(223)
plt.imshow(im2w/255.0)

plt.show()

In [ ]:
data_path = '/store/dataset/vdenoising/vimeo/input1/v1/frame_017_noisy.png'


from PIL import Image
import numpy as np
img_pil = Image.open(data_path)
print(img_pil)
img = np.array(img_pil)
print(img.shape)
img = img[:,:,::-1]
print(img.shape)
from matplotlib import pyplot as plt
plt.imshow(img)
plt.show()

from skimage import filters
gaussian_img = filters.gaussian(img, sigma=1.5)

plt.imshow(gaussian_img)
plt.show()


#median_img = filters.median(img, np.ones((3, 3)))
#plt.imshow(median_img)
#plt.show()

In [ ]:
a = np.array([1,2,3])

a[::-1]

import cv2
img = cv2.imread(data_path)
plt.imshow(img)
plt.show()


blur = cv2.medianBlur(img,5)
plt.imshow(blur)
plt.show()


In [ ]:
import math
import numbers
import torch
from torch import nn
from torch.nn import functional as F

class GaussianSmoothing(nn.Module):
    """
    Apply gaussian smoothing on a
    1d, 2d or 3d tensor. Filtering is performed seperately for each channel
    in the input using a depthwise convolution.
    Arguments:
        channels (int, sequence): Number of channels of the input tensors. Output will
            have this number of channels as well.
        kernel_size (int, sequence): Size of the gaussian kernel.
        sigma (float, sequence): Standard deviation of the gaussian kernel.
        dim (int, optional): The number of dimensions of the data.
            Default value is 2 (spatial).
    """
    def __init__(self, channels, kernel_size=3, sigma=1.5, padding=1, dim=2):
        super(GaussianSmoothing, self).__init__()
        if isinstance(kernel_size, numbers.Number):
            kernel_size = [kernel_size] * dim
        if isinstance(sigma, numbers.Number):
            sigma = [sigma] * dim

        self.padding = padding

        # print(kernel_size, sigma)
        # The gaussian kernel is the product of the
        # gaussian function of each dimension.
        kernel = 1
        meshgrids = torch.meshgrid(
            [
                torch.arange(size, dtype=torch.float32)
                for size in kernel_size
            ]
        )

        for size, std, mgrid in zip(kernel_size, sigma, meshgrids):
            mean = (size - 1) / 2

            kernel *= 1 / (std * math.sqrt(2 * math.pi)) * \
                      torch.exp(-((mgrid - mean) / (2 * std)) ** 2)

        # Make sure sum of values in gaussian kernel equals 1.
        kernel = kernel / torch.sum(kernel)


        # Reshape to depthwise convolutional weight
        kernel = kernel.view(1, 1, *kernel.size())
        kernel = kernel.repeat(channels, *[1] * (kernel.dim() - 1))
        #print(kernel.shape)
        self.register_buffer('weight', kernel)
        self.groups = channels

        if dim == 1:
            self.conv = F.conv1d
        elif dim == 2:
            self.conv = F.conv2d
        elif dim == 3:
            self.conv = F.conv3d
        else:
            raise RuntimeError(
                'Only 1, 2 and 3 dimensions are supported. Received {}.'.format(dim)
            )

    def forward(self, input):
        """
        Apply gaussian filter to input.
        Arguments:
            input (torch.Tensor): Input to apply gaussian filter on.
        Returns:
            filtered (torch.Tensor): Filtered output.
        """
        return self.conv(input, weight=self.weight, padding=self.padding, groups=self.groups)

smoothing = GaussianSmoothing(2, 5, 1.5, padding=2, dim=2)
input = torch.rand(3, 2, 2, 100, 100)

f, b, c, h, w = input.size()
print(input.shape)

input = input.view(f*b, c, h, w)
print(input.shape)
#input = F.pad(input, (2, 2, 2, 2), mode='reflect')
output = smoothing(input)

len(output.chunk(2, dim=2))

t1, t2 = output.chunk(2, dim=2)

print(t1.shape)
print(t2.shape)

#output = output.view(f, b, c, h, w)
#print(output.shape)

In [ ]:
img = cv2.imread(data_path) / 255.0
plt.imshow(img)
plt.show()

import torch
img_tensor = torch.tensor(img).permute([2,0,1]).unsqueeze(0).type(torch.FloatTensor)

smoothing = GaussianSmoothing(3, 7, 2, padding=2, dim=2)
s_img_tensor = smoothing(img_tensor)

s_img = s_img_tensor.squeeze().permute(1,2,0)
plt.imshow(s_img)
plt.show()

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.utils import _pair, _quadruple


class MedianPool2d(nn.Module):
    """ Median pool (usable as median filter when stride=1) module.
    
    Args:
         kernel_size: size of pooling kernel, int or 2-tuple
         stride: pool stride, int or 2-tuple
         padding: pool padding, int or 4-tuple (l, r, t, b) as in pytorch F.pad
         same: override padding and enforce same padding, boolean
    """
    def __init__(self, kernel_size=3, stride=1, padding=1, same=False):
        super(MedianPool2d, self).__init__()
        self.k = _pair(kernel_size)
        self.stride = _pair(stride)
        self.padding = _quadruple(padding)  # convert to l, r, t, b
        self.same = same

    def _padding(self, x):
        if self.same:
            ih, iw = x.size()[2:]
            if ih % self.stride[0] == 0:
                ph = max(self.k[0] - self.stride[0], 0)
            else:
                ph = max(self.k[0] - (ih % self.stride[0]), 0)
            if iw % self.stride[1] == 0:
                pw = max(self.k[1] - self.stride[1], 0)
            else:
                pw = max(self.k[1] - (iw % self.stride[1]), 0)
            pl = pw // 2
            pr = pw - pl
            pt = ph // 2
            pb = ph - pt
            padding = (pl, pr, pt, pb)
        else:
            padding = self.padding
        return padding
    
    def forward(self, x):
        # using existing pytorch functions and tensor ops so that we get autograd, 
        # would likely be more efficient to implement from scratch at C/Cuda level
        x = F.pad(x, self._padding(x), mode='reflect')
        x = x.unfold(2, self.k[0], self.stride[0]).unfold(3, self.k[1], self.stride[1])
        x = x.contiguous().view(x.size()[:4] + (-1,)).median(dim=-1)[0]
        return x

    
img = cv2.imread(data_path) / 255.0
plt.imshow(img)
plt.show()

import torch
img_tensor = torch.tensor(img).permute([2,0,1]).unsqueeze(0).type(torch.FloatTensor)

smoothing = MedianPool2d(kernel_size=5, padding=2)
s_img_tensor = smoothing(img_tensor)

print(s_img_tensor.shape)

s_img = s_img_tensor.squeeze().permute(1,2,0)
plt.imshow(s_img)
plt.show()

In [ ]:
import GPUtil
import time

while(True):
    try:
        DEVICE_ID_LIST = GPUtil.getFirstAvailable()
        break;
    except:
        time.sleep(1)
        print("hello world")

In [ ]:
import os
command = 'python main.py'
os.system(command)

In [ ]:
import glob
import os
import torch
import imageio

dataroot = "/store/dataset/vdenoising/vimeo/v1/"
def load_video_data(data_dir):
    noised_paths = glob.glob(os.path.join(data_dir, "noised", "*.png"))
    target_paths = glob.glob(os.path.join(data_dir, "target", "*.png"))
    noised_paths.sort()
    target_paths.sort()
    noised_images = []
    target_images = []
    for noised_path, target_path in zip(noised_paths, target_paths):
        noised_images.append(torch.tensor(imageio.imread(noised_path)))
        target_images.append(torch.tensor(imageio.imread(target_path)))

    ##frames, batch, CHW
    noised_images_torch = torch.stack(noised_images, dim=0).unsqueeze(1).permute(0,1,4,2,3)
    target_images_torch = torch.stack(target_images, dim=0).unsqueeze(1).permute(0,1,4,2,3)

    return noised_images_torch.type(torch.FloatTensor), target_images_torch.type(torch.FloatTensor)


#dirdata = '/home/lrh/git/data/v2'
dirdata = '/store/dataset/vdenoising/vimeo/v1/'
noise_input, target = load_video_data(dirdata)

print(noise_input.shape)

In [ ]:
## load_video_data
    # Return a generator for generating sequence of video frames
    # Indefinite length according to a file recording the pivot frame point
import os
pointpath = os.path.join(dataroot, 'point.txt')
f = open(pointpath)
points = [int(line) for line in f.readlines()]


from matplotlib import pyplot as plt

prepoint = 0
for point in points:
    print("frame index:", prepoint, point)
    img = noise_input[point - 1]

    img = img[0].permute(1,2,0).numpy() / 255.0
    img = img[:,:,::-1]

    plt.subplot(121)
    plt.imshow(img)
    
    
    img = noise_input[prepoint]

    img = img[0].permute(1,2,0).numpy() / 255.0
    img = img[:,:,::-1]
    plt.subplot(122)
    plt.imshow(img)
    plt.show()
    
    prepoint = point

In [ ]:
img = noise_input[10]

img = img[0].permute(1,2,0).numpy() / 255.0
img = img[:,:,::-1]

plt.imshow(img)

In [6]:
import glob
import os
import torch
import imageio

dataroot = "/store/dataset/vdenoising/vimeo/v1/"

def load_video_data(data_dir, points = None):
    prepoint = 0
    if points is None:
        pointpath = os.path.join(data_dir, 'point.txt')
        f = open(pointpath)
        points = [int(line) for line in f.readlines()]
        
    noised_paths = glob.glob(os.path.join(data_dir, "noised", "*.png"))
    target_paths = glob.glob(os.path.join(data_dir, "target", "*.png"))
    noised_paths.sort()
    target_paths.sort()
    for point in points:
        n_paths = noised_paths[prepoint:point-1]
        t_paths = target_paths[prepoint:point-1]
        noised_images = []
        target_images = []
        for noised_path, target_path in zip(n_paths, t_paths):
            noised_images.append(torch.tensor(imageio.imread(noised_path)))
            target_images.append(torch.tensor(imageio.imread(target_path)))
        ##frames, batch, CHW
        noised_images_torch = torch.stack(noised_images, dim=0).unsqueeze(1).permute(0,1,4,2,3).type(torch.FloatTensor)
        target_images_torch = torch.stack(target_images, dim=0).unsqueeze(1).permute(0,1,4,2,3).type(torch.FloatTensor)
        
        yield noised_images_torch, target_images_torch, [prepoint, point-1]
        
        prepoint = point


data_generator = load_video_data(dataroot)
for d in data_generator:
    print(d[0][0])

tensor([[[[  0.,  18.,  29.,  ...,  43.,  29.,  21.],
          [  0.,  20.,  23.,  ...,  26.,  23.,   9.],
          [  6.,  11.,  21.,  ...,  38.,  25.,   9.],
          ...,
          [104., 106., 146.,  ...,  98.,  98., 113.],
          [125., 125., 139.,  ..., 100., 106., 113.],
          [ 33., 119., 130.,  ...,  98.,  96.,  99.]],

         [[  2.,  25.,  25.,  ...,  38.,  25.,  12.],
          [  0.,  11.,  15.,  ...,  20.,  17.,   1.],
          [  0.,   0.,  12.,  ...,  29.,  16.,   0.],
          ...,
          [111., 110., 128.,  ...,  76.,  73.,  79.],
          [128., 128., 129.,  ...,  80.,  83.,  80.],
          [ 36., 120., 125.,  ...,  85.,  80.,  84.]],

         [[  5.,  23.,  21.,  ...,  32.,  19.,  10.],
          [  0.,  10.,   8.,  ...,  10.,   7.,   0.],
          [  0.,   0.,   6.,  ...,  22.,   8.,   0.],
          ...,
          [112., 107., 108.,  ...,  66.,  81.,  84.],
          [129., 125., 118.,  ...,  74.,  89.,  85.],
          [ 37., 112., 120.,  ...

tensor([[[[ 18.,  23., 142.,  ...,   0.,   0.,   0.],
          [  3.,   9., 110.,  ...,   9.,   9.,   9.],
          [  1.,   3., 102.,  ...,   7.,  27.,  20.],
          ...,
          [  0.,   0.,   6.,  ...,  23.,  34.,  49.],
          [  9.,   0.,   4.,  ...,  23.,  29.,   9.],
          [ 23.,   0.,   6.,  ...,  24.,  24.,  17.]],

         [[ 13.,  19., 155.,  ...,   0.,   0.,   0.],
          [  1.,  13., 134.,  ...,  11.,   8.,   7.],
          [  4.,  17., 141.,  ...,  12.,  28.,  18.],
          ...,
          [  0.,   0.,   7.,  ...,  28.,  24.,  40.],
          [ 10.,   0.,   7.,  ...,  26.,  23.,   3.],
          [ 23.,   0.,   7.,  ...,  27.,  27.,  14.]],

         [[  8.,  15., 148.,  ...,   0.,   0.,   0.],
          [  0.,   1., 117.,  ...,   3.,  10.,  14.],
          [  0.,   0., 106.,  ...,   1.,  19.,  21.],
          ...,
          [  0.,   1.,   8.,  ...,  30.,  21.,  30.],
          [ 13.,   1.,   9.,  ...,  25.,  22.,   1.],
          [ 27.,   2.,   6.,  ...

tensor([[[[ 24.,  17.,  21.,  ...,  31.,  33.,  25.],
          [ 36.,  16.,  19.,  ...,  25.,  34.,  22.],
          [ 36.,  16.,   5.,  ...,  27.,  24.,  21.],
          ...,
          [ 70.,  63.,  63.,  ...,  49.,  49.,  37.],
          [ 55.,  63.,  63.,  ...,  55.,  60.,  56.],
          [ 59.,  90.,  90.,  ...,  55.,  55.,  63.]],

         [[ 25.,  19.,  30.,  ...,  34.,  34.,  26.],
          [ 38.,  19.,  30.,  ...,  27.,  33.,  23.],
          [ 38.,  19.,  21.,  ...,  23.,  22.,  23.],
          ...,
          [ 79.,  83.,  90.,  ...,  58.,  58.,  40.],
          [ 65.,  82.,  89.,  ...,  58.,  63.,  58.],
          [ 63., 103., 104.,  ...,  57.,  57.,  65.]],

         [[ 21.,  18.,  29.,  ...,  30.,  43.,  30.],
          [ 28.,  18.,  34.,  ...,  23.,  42.,  32.],
          [ 29.,  20.,  26.,  ...,  21.,  28.,  30.],
          ...,
          [ 77.,  74.,  79.,  ...,  49.,  49.,  38.],
          [ 62.,  77.,  81.,  ...,  51.,  56.,  54.],
          [ 60.,  99., 100.,  ...

tensor([[[[106.,  88.,  75.,  ...,  67., 102., 122.],
          [ 96.,  82.,  77.,  ...,  64.,  72., 128.],
          [ 90.,  88.,  81.,  ...,  61.,  61., 128.],
          ...,
          [ 79.,  68.,  74.,  ...,  24.,  23.,  23.],
          [ 92.,  71.,  70.,  ...,  21.,  11.,  10.],
          [140.,  87.,  77.,  ...,  17.,  15.,   0.]],

         [[128.,  94.,  80.,  ...,  76., 120., 148.],
          [ 94.,  83.,  85.,  ...,  78.,  92., 146.],
          [ 89.,  86.,  90.,  ...,  84.,  83., 147.],
          ...,
          [ 92.,  97., 108.,  ...,  32.,  31.,  31.],
          [112., 100., 109.,  ...,  28.,  18.,  16.],
          [142.,  95.,  95.,  ...,  21.,  18.,   0.]],

         [[123.,  79.,  68.,  ...,  63., 113., 147.],
          [ 43.,  48.,  57.,  ...,  57.,  75., 140.],
          [ 37.,  51.,  52.,  ...,  44.,  48., 132.],
          ...,
          [100., 105., 117.,  ...,  24.,  28.,  28.],
          [118., 108., 118.,  ...,  24.,  15.,  16.],
          [120.,  88., 115.,  ...

tensor([[[[49., 27., 26.,  ..., 20., 20., 19.],
          [33., 31., 32.,  ..., 20., 21., 22.],
          [23., 23., 31.,  ..., 21., 21., 30.],
          ...,
          [52., 52., 52.,  ...,  4., 18.,  5.],
          [49., 47., 41.,  ...,  5., 13.,  0.],
          [29., 47., 41.,  ..., 28., 28., 11.]],

         [[52., 33., 35.,  ..., 52., 53., 41.],
          [37., 37., 38.,  ..., 24., 27., 27.],
          [26., 29., 39.,  ..., 10., 11., 25.],
          ...,
          [48., 48., 56.,  ..., 24., 36., 42.],
          [48., 46., 49.,  ..., 23., 31., 34.],
          [28., 46., 48.,  ..., 36., 37., 29.]],

         [[46., 24., 21.,  ..., 52., 47., 38.],
          [26., 23., 18.,  ..., 26., 21., 22.],
          [19., 17., 19.,  ...,  6.,  5., 21.],
          ...,
          [44., 42., 47.,  ..., 11., 24., 33.],
          [45., 43., 45.,  ..., 12., 20., 27.],
          [23., 44., 47.,  ..., 33., 28., 27.]]]])
tensor([[[[152., 151., 126.,  ...,  56.,  60.,  37.],
          [131., 123., 123.,  

tensor([[[[ 41.,  56.,  61.,  ..., 106., 156., 104.],
          [ 36.,  43.,  58.,  ..., 144., 145., 152.],
          [ 66.,  37.,  35.,  ..., 127., 132., 139.],
          ...,
          [ 59.,  47.,  33.,  ...,  47.,  55.,  53.],
          [ 73.,  50.,  42.,  ...,  50.,  50.,  53.],
          [ 75.,  57.,  44.,  ...,  50.,  50.,  41.]],

         [[ 44.,  65.,  63.,  ..., 200., 177., 120.],
          [ 44.,  50.,  60.,  ..., 181., 182., 190.],
          [ 71.,  53.,  53.,  ..., 177., 181., 195.],
          ...,
          [ 65.,  58.,  56.,  ...,  47.,  57.,  61.],
          [ 77.,  57.,  57.,  ...,  55.,  55.,  60.],
          [ 78.,  61.,  56.,  ...,  54.,  54.,  47.]],

         [[ 29.,  57.,  57.,  ..., 191., 181., 122.],
          [ 44.,  51.,  55.,  ..., 187., 187., 194.],
          [ 36.,  54.,  58.,  ..., 178., 190., 201.],
          ...,
          [ 59.,  50.,  43.,  ...,  35.,  47.,  55.],
          [ 69.,  51.,  50.,  ...,  51.,  51.,  60.],
          [ 62.,  56.,  50.,  ...

tensor([[[[ 95.,  87.,  90.,  ...,  47.,  74.,  34.],
          [ 94.,  84.,  92.,  ...,  61.,  58.,  26.],
          [ 90.,  85.,  69.,  ...,  53.,  52.,  42.],
          ...,
          [ 27.,   8.,   5.,  ...,  33.,  36.,  40.],
          [ 29.,  20.,  15.,  ...,  33.,  35.,  37.],
          [  0.,  20.,  26.,  ...,  13.,  13.,  46.]],

         [[106., 109., 111.,  ...,  91.,  71.,  20.],
          [107., 106., 116.,  ...,  77.,  68.,  23.],
          [101., 107.,  93.,  ...,  83.,  71.,  47.],
          ...,
          [ 43.,  31.,  26.,  ...,  42.,  43.,  44.],
          [ 37.,  41.,  27.,  ...,  42.,  41.,  43.],
          [  2.,  38.,  36.,  ...,  20.,  20.,  57.]],

         [[ 92., 101., 103.,  ...,  97.,  78.,  23.],
          [ 88.,  81.,  91.,  ...,  89.,  71.,  30.],
          [ 80.,  82.,  55.,  ...,  81.,  71.,  46.],
          ...,
          [ 38.,  24.,  17.,  ...,  29.,  35.,  39.],
          [ 32.,  35.,  21.,  ...,  32.,  32.,  37.],
          [  0.,  37.,  30.,  ...